Using data from Kevin Ummel and the American Community Survey Supplemental Poverty Measure research file.

In [104]:
import numpy as np
import pandas as pd
import plotly.express as px
import microdf as mdf
from ubicenter import format_fig
import us
import pyreadr

# Remove when exporting and creating in vscode.
import plotly.io as pio

pio.renderers.default = "notebook"

cu = pd.read_csv("../data/carbon_footprint_2018.csv")
cuid_sporder = pd.DataFrame(
    list(pyreadr.read_r("../data/CUID-SPORDER.rds").values())[0]
)
p = pd.read_csv("../data/acs_poverty.csv.gz")

LABELS = dict(
    price="Carbon price",
    poverty_chg="Change to poverty rate",
    deep_poverty_chg="Change to deep poverty rate",
    age_group="Age group",
    state="State",
    poverty_str = "",
    deep_poverty_str = "",
    base_poverty="Current poverty rate",
    base_deep_poverty="Current deep poverty rate",
    poverty="Reform poverty rate",
    deep_poverty="Reform deep poverty rate",
)


In [105]:
# Per Kevin Ummel, serialno is first 13 characters of CUID
def cuid2serialno(cuid):
    return cuid.str[6:13].astype(int)


cuid_sporder["serialno"] = cuid2serialno(cuid_sporder.CUID)
cu["serialno"] = cuid2serialno(cu.CUID)
# Household dataset formed from the CU dataset.
h_co2 = cu.drop(columns="CUID").groupby("serialno").sum().reset_index()
# Columns to rename as they're aggregated to household.
h_co2_cols = h_co2.columns.drop("serialno")
h_co2.rename(
    columns=dict(zip(h_co2_cols, [i + "_hh" for i in h_co2_cols])),
    inplace=True,
)
# Assign emissions equally across household members.
p_per_h = (
    p.groupby("serialno")
    .size()
    .reset_index()
    .rename(columns={0: "people_in_hh"})
)
p = p.merge(p_per_h, on="serialno").merge(h_co2, on="serialno")
for i in h_co2_cols:
    p[i] = p[i + "_hh"] / p.people_in_hh

SPMU_COLS = ["id", "povthreshold", "resources"]
p["person"] = 1
s = (
    p.groupby(["spm_" + i for i in SPMU_COLS])[["person", "tco2"]]
    .sum()
    .reset_index()
)

total_co2 = mdf.weighted_sum(p, "tco2", "wt")
total_pop = p.wt.sum()
co2_pp = total_co2 / total_pop
p["age_group"] = np.where(p.age < 18, "Child", "Adult")



In [129]:
def carbon_dividend(price):
    def pov_metrics(p):
        poverty = mdf.poverty_rate(
            p, "spm_resources_new", "spm_povthreshold", "wt",
        )
        deep_poverty = mdf.deep_poverty_rate(
            p, "spm_resources_new", "spm_povthreshold", "wt",
        )
        return pd.Series(dict(poverty=poverty, deep_poverty=deep_poverty))

    dividend = co2_pp * price
    s["dividend"] = s.person * dividend
    s["co2_tax"] = s.tco2 * price
    s["spm_resources_new"] = s.spm_resources + s.dividend - s.co2_tax
    p2 = p.merge(s[["spm_id", "spm_resources_new"]], on="spm_id")
    pov_overall = pd.DataFrame(pov_metrics(p2)).T
    pov_overall["age_group"] = "All"
    pov_overall["st"] = 0
    pov_age = p2.groupby("age_group").apply(pov_metrics).reset_index()
    pov_age["st"] = 0
    pov_state = p2.groupby("st").apply(pov_metrics).reset_index()
    pov_state["age_group"] = "All"
    pov_age_state = p2.groupby(["st", "age_group"]).apply(pov_metrics).reset_index()
    res = pd.concat([pov_overall, pov_age, pov_state, pov_age_state], axis=0)
    res["price"] = price
    return res


In [131]:
carbon_price = pd.concat(
    [carbon_dividend(i) for i in np.arange(0, 15, 5)]
).reset_index(drop=True)
base_poverty = (
    carbon_price[carbon_price.price == 0]
    .rename(
        columns={
            "poverty": "base_poverty",
            "deep_poverty": "base_deep_poverty",
        }
    )
    .drop(columns="price")
)
carbon_price = carbon_price.merge(base_poverty, on=["age_group", "st"])
carbon_price["poverty_chg"] = (
    carbon_price.poverty / carbon_price.base_poverty - 1
)
carbon_price["deep_poverty_chg"] = (
    carbon_price.deep_poverty / carbon_price.base_deep_poverty - 1
)


def get_state_info(st):
    fips = str(int(st)).zfill(2)
    if st > 0:
        state = us.states.lookup(fips)
        abbr = state.abbr
        name = state.name
    else:
        abbr = "US"
        name = "US"
    return pd.Series(dict(state_abbr=abbr, state_name=name))


carbon_price[["state_abbr", "state_name"]] = carbon_price.st.apply(
    get_state_info
)
carbon_price


,poverty,deep_poverty,age_group,st,price,base_poverty,base_deep_poverty,poverty_chg,deep_poverty_chg
0,0.153290,0.056553,All,0.0,0,0.153290,0.056553,0.000000,0.000000
1,0.152075,0.055970,All,0.0,5,0.153290,0.056553,-0.007922,-0.010308
2,0.150823,0.055417,All,0.0,10,0.153290,0.056553,-0.016091,-0.020088
3,0.147295,0.056510,Adult,0.0,0,0.147295,0.056510,0.000000,0.000000
4,0.146466,0.056110,Adult,0.0,5,0.147295,0.056510,-0.005627,-0.007079
...,...,...,...,...,...,...,...,...,...
463,0.111470,0.057945,Adult,56.0,5,0.112688,0.058307,-0.010809,-0.006223
464,0.111793,0.057850,Adult,56.0,10,0.112688,0.058307,-0.007938,-0.007849
465,0.117119,0.045608,Child,56.0,0,0.117119,0.045608,0.000000,0.000000
466,0.112023,0.042797,Child,56.0,5,0.117119,0.045608,-0.043515,-0.061652


In [138]:
# Overall poverty and deep poverty.
fig = px.line(
    carbon_price[(carbon_price.age_group == "All") & (carbon_price.state_abbr == "US")],
    "price",
    ["poverty_chg", "deep_poverty_chg"],
    title="Carbon dividend poverty impact",
    labels=LABELS,
)
fig.update_layout(xaxis_tickformat="$", yaxis_tickformat="%")
format_fig(fig)


In [139]:
fig = px.bar(
    carbon_price[(carbon_price.state_abbr == "US") & (carbon_price.price > 0)],
    "age_group",
    ["poverty_chg", "deep_poverty_chg"],
    animation_frame="price",
    title="Carbon dividend poverty impact by age",
    labels=LABELS,
)
fig.update_layout(yaxis_tickformat="%", barmode="group")
format_fig(fig)


In [147]:
map_data.columns

Index(['poverty', 'deep_poverty', 'age_group', 'st', 'price', 'base_poverty',
       'base_deep_poverty', 'poverty_chg', 'deep_poverty_chg', 'state_abbr',
       'state_name'],
      dtype='object')

In [163]:
map_data = carbon_price[
    (carbon_price.state_abbr != "US")
    & (carbon_price.age_group == "All")
    & (carbon_price.price > 0)
].copy()


def pov_str(name, base, new):
    fall = 1 - (new / base)
    return f"{name} falls {fall:.0%} from {base:.1%} to {new:.1%}"


map_data["poverty_string"] = map_data.apply(
    lambda x: pov_str("Poverty", x.base_poverty, x.poverty), axis=1
)
map_data["deep_poverty_string"] = map_data.apply(
    lambda x: pov_str("Deep poverty", x.base_deep_poverty, x.deep_poverty),
    axis=1,
)
fig = px.choropleth(
    map_data,
    locations="state_abbr",
    locationmode="USA-states",
    color="poverty_chg",
    scope="usa",
    animation_frame="price",
    title="Poverty impact of a carbon dividend",
    labels=LABELS,
    hover_name="state_name",
    # hover_data=dict(
    #     state_abbr=False,
    #     price=False,
    #     poverty_chg=False,
    #     poverty_string=True,
    #     deep_poverty_string=True,
    # ),
    custom_data=["state_name", "poverty_string", "deep_poverty_string"],
)
fig.update_traces(hovertemplate="<b>%{customdata[0]}</b><br>%{customdata[1]}<br>%{customdata[2]}")
fig.update_layout(coloraxis_showscale=False)  # , colorscale='Blues')
format_fig(fig)



In [97]:
map_data

,poverty,deep_poverty,age_group,state,price,base_poverty,base_deep_poverty,poverty_chg,deep_poverty_chg
10,0.170265,0.068347,All,Alabama,5,0.170629,0.069072,-0.002129,-0.010504
11,0.169259,0.067446,All,Alabama,10,0.170629,0.069072,-0.008024,-0.023538
13,0.124447,0.041596,All,Alaska,5,0.124537,0.041958,-0.000723,-0.008619
14,0.122194,0.041350,All,Alaska,10,0.124537,0.041958,-0.018813,-0.014488
16,0.148776,0.056021,All,Arizona,5,0.150618,0.056761,-0.012230,-0.013035
...,...,...,...,...,...,...,...,...,...
155,0.159037,0.060438,All,West Virginia,10,0.161098,0.062352,-0.012793,-0.030691
157,0.105142,0.041522,All,Wisconsin,5,0.105884,0.041547,-0.007012,-0.000582
158,0.104086,0.041469,All,Wisconsin,10,0.105884,0.041547,-0.016983,-0.001881
160,0.111598,0.054427,All,Wyoming,5,0.113717,0.055358,-0.018632,-0.016829


## Analysis

Total CO2

Compare to 6.6 billion tons CO2 equivalent ([EPA](https://www.epa.gov/ghgemissions/inventory-us-greenhouse-gas-emissions-and-sinks)), though includes other GHGs.

In [4]:
total_co2 / 1e9

6.884031485652741

In [5]:
mdf.poverty_rate(p, "spm_resources", "spm_povthreshold", "wt")

0.1532896866553634

In [6]:
mdf.poverty_rate(p, "spm_resources_new", "spm_povthreshold", "wt")

0.15018853668734483

In [10]:
1 - mdf.poverty_rate(p, "spm_resources_new", "spm_povthreshold", "wt") / mdf.poverty_rate(p, "spm_resources", "spm_povthreshold", "wt")

0.020230649795708655

In [11]:
1 - mdf.deep_poverty_rate(p, "spm_resources_new", "spm_povthreshold", "wt") / mdf.deep_poverty_rate(p, "spm_resources", "spm_povthreshold", "wt")

0.024789704725417305